In [656]:
# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm



# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Librerias para generar gráficas
# ----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


En este notebook se limpian los archivos de nivel educativivo alcanzado extraidos del INE. Por un lado se tratan los municipios (grandes y pequeños) y por otro las provincias, comunidades y total de país, pues provienen de archivos distintos.

## Limpiamos y concatenamos los dataframes de nivel educativo de municipios pequeños (de 50 a 500 habitantes) y grandes (más de 500 habitantes)

In [657]:
df_nivel_educativo_municipios_pequenios = pd.read_csv("datos/ine/nivel_educativo_municipios_pequenios.csv",sep=';')

In [658]:
df_nivel_educativo_municipios_pequenios.head()

,Municipios,Sexo,Nivel de estudios,Periodo,Total
0,01006 Armiñón,Total,Total,2022,205
1,01006 Armiñón,Total,Total,2021,198
2,01006 Armiñón,Total,Educación primaria e inferior,2022,18
3,01006 Armiñón,Total,Educación primaria e inferior,2021,17
4,01006 Armiñón,Total,Primera etapa de educación secundaria y similar,2022,55


In [659]:
df_nivel_educativo_municipios_grandes = pd.read_csv("datos/ine/nivel_educativo_municipios_grandes.csv",sep=';')

In [660]:
df_nivel_educativo_municipios_grandes.sample(20)

,Municipios,Sexo,País de nacimiento,Nivel de estudios,Periodo,Total
507935,31009 Aibar/Oibar,Total,España,Doctorado universitario,2021,4
43528,04029 Berja,Mujeres,Extranjera,"Enseñanzas de formación profesional, artes plá...",2022,17
208063,12138 Vinaròs,Mujeres,Extranjera,"Segunda etapa de educación secundaria, con ori...",2021,397.0
262622,16148 Palomares del Campo,Hombres,Total,"Segunda etapa de educación secundaria, con ori...",2022,22.0
234474,14056 Puente Genil,Total,Extranjera,Doctorado universitario,2022,1.0
741636,46124 Fontanars dels Alforins,Hombres,España,Grados universitarios de más de 240 créditos E...,2022,20.0
126455,08157 Pallejà,Hombres,España,Doctorado universitario,2021,47
801082,49239 Villabuena del Puente,Mujeres,Extranjera,"Másteres, especialidades en Ciencias de la Sal...",2022,0.0
65247,06025 Calamonte,Hombres,Extranjera,Grados universitarios de más de 240 créditos E...,2021,3
186385,11002 Alcalá del Valle,Hombres,Total,Total,2021,2.223


In [661]:
df_nivel_estudios_en_curso_por_lugar = pd.read_csv("datos/ine/nivel_estudios_en_curso_por_lugar.csv",sep=';')

In [662]:
df_nivel_estudios_en_curso_por_lugar.sample(20)

,Capitales y grandes ciudades,Sexo,Relación entre lugar de residencia y lugar de estudios,Nacionalidad,Nivel de estudios en curso,Periodo,Total
136737,44216 Teruel,Hombres,No cursa estudios,Extranjera,Total,2021,1.344
76922,28022 Boadilla del Monte,Total,No cursa estudios,Española,Educación superior,2022,0
2994,03009 Alcoi/Alcoy,Mujeres,No cursa estudios,Española,Educación primaria e inferior,2022,0
4367,03031 Benidorm,Total,No cursa estudios,Extranjera,Cursa estudios pero no hay información sobre l...,2021,0
117314,35022 Santa Lucía de Tirajana,Hombres,Mismo municipio al de residencia,Total,Educación primaria e inferior,2022,48
62488,20069 Donostia/San Sebastián,Mujeres,No cursa estudios,Extranjera,"Segunda etapa de Educación Secundaria, con ori...",2022,0
26900,08121 Mataró,Mujeres,Total,Española,Primera etapa de Educación Secundaria y similar,2022,136
138677,45168 Toledo,Hombres,Cursa estudios pero no hay información sobre e...,Total,Primera etapa de Educación Secundaria y similar,2021,1.0
29411,08184 Rubí,Total,Distinta provincia de la misma comunidad,Extranjera,Educación primaria e inferior,2021,0
34631,08266 Cerdanyola del Vallès,Hombres,Total,Española,"Segunda etapa de educación secundaria, con ori...",2021,712


In [663]:
df_nivel_educativo_municipios_pequenios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102570 entries, 0 to 102569
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Municipios         102570 non-null  object
 1   Sexo               102570 non-null  object
 2   Nivel de estudios  102570 non-null  object
 3   Periodo            102570 non-null  int64 
 4   Total              102570 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 3.9+ MB


In [664]:
df_nivel_educativo_municipios_grandes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820512 entries, 0 to 820511
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Municipios          820512 non-null  object
 1   Sexo                820512 non-null  object
 2   País de nacimiento  820512 non-null  object
 3   Nivel de estudios   820512 non-null  object
 4   Periodo             820512 non-null  int64 
 5   Total               820512 non-null  object
dtypes: int64(1), object(5)
memory usage: 37.6+ MB


Formateamos la columna "Total"

In [665]:
# Eliminar caracteres no numéricos y convertir la columna 'Total' a entero
df_nivel_educativo_municipios_grandes['Total'] = (
    df_nivel_educativo_municipios_grandes['Total']
    .astype(str)  # Convertir a string para realizar operaciones
    .str.replace(r'[^\d]', '', regex=True)  # Eliminar caracteres no numéricos
    .astype(int)  # Convertir a entero
)

# Confirmar el cambio mostrando las primeras filas
df_nivel_educativo_municipios_grandes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820512 entries, 0 to 820511
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Municipios          820512 non-null  object
 1   Sexo                820512 non-null  object
 2   País de nacimiento  820512 non-null  object
 3   Nivel de estudios   820512 non-null  object
 4   Periodo             820512 non-null  int64 
 5   Total               820512 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.6+ MB


Queremos juntar los datos de nivel educativo alcanzado en un solo dataframe. 

In [666]:
unique_grandes = df_nivel_educativo_municipios_grandes["Nivel de estudios"].unique()
unique_pequenios = df_nivel_educativo_municipios_pequenios["Nivel de estudios"].unique()

unique_grandes, unique_pequenios

(array(['Total', 'Sin estudios', 'Educación primaria',
        'Primera etapa de educación secundaria y similar',
        'Segunda etapa de educación secundaria, con orientación general',
        'Segunda etapa de educación secundaria con orientación profesional, Educación postsecundaria no superior',
        'Enseñanzas de formación profesional, artes plásticas y diseño y deportivas de grado superior y equivalentes',
        'Grados universitarios de hasta 240 créditos ECTS, diplomados universitarios y equivalentes',
        'Grados universitarios de más de 240 créditos ECTS, licenciados y equivalentes',
        'Másteres, especialidades en Ciencias de la Salud por el sistema de residencia y similares',
        'Doctorado universitario'], dtype=object),
 array(['Total', 'Educación primaria e inferior',
        'Primera etapa de educación secundaria y similar',
        'Segunda etapa de educación secundaria y educación postsecundaria no superior',
        'Educación superior'], dtype=o

Observamos que la columna "Nivel de estudios" toma distintos valores en ambos dataframes, pero podemos hacer un mapeo, mediante un diccionario que permita asignar los mismos valores a dicha columna para ambos dataframes.

In [667]:
def reasignar_nivel_estudios(df, columna):
    """
    Reasigna los valores de una columna específica en un DataFrame 
    según un diccionario de mapeo predefinido.

    Args:
    df (pd.DataFrame): DataFrame que contiene la columna a modificar.
    columna (str): Nombre de la columna cuyos valores serán reasignados.

    Returns:
    pd.DataFrame: DataFrame con los valores de la columna reasignados.
    """
    # Crear un diccionario de mapeo para reasignar los valores
    mapeo_nivel_estudios = {
        'Total': 'Total',
        'Sin estudios': 'Educación primaria e inferior',
        'Educación primaria': 'Educación primaria e inferior',
        'Primera etapa de educación secundaria y similar': 'Primera etapa de educación secundaria y similar',
        'Segunda etapa de educación secundaria, con orientación general': 'Segunda etapa de educación secundaria y educación postsecundaria no superior',
        'Segunda etapa de educación secundaria con orientación profesional, Educación postsecundaria no superior': 'Segunda etapa de educación secundaria y educación postsecundaria no superior',
        'Enseñanzas de formación profesional, artes plásticas y diseño y deportivas de grado superior y equivalentes': 'Educación superior',
        'Grados universitarios de hasta 240 créditos ECTS, diplomados universitarios y equivalentes': 'Educación superior',
        'Grados universitarios de más de 240 créditos ECTS, licenciados y equivalentes': 'Educación superior',
        'Másteres, especialidades en Ciencias de la Salud por el sistema de residencia y similares': 'Educación superior',
        'Doctorado universitario': 'Educación superior'
    }

    # Reasignar los valores en la columna especificada
    df[columna] = df[columna].replace(mapeo_nivel_estudios)
    
    return df




In [668]:
df_nivel_educativo_municipios_grandes = reasignar_nivel_estudios(
    df_nivel_educativo_municipios_grandes, 'Nivel de estudios'
)

In [669]:
df_nivel_educativo_municipios_grandes['Nivel de estudios'].unique()

array(['Total', 'Educación primaria e inferior',
       'Primera etapa de educación secundaria y similar',
       'Segunda etapa de educación secundaria y educación postsecundaria no superior',
       'Educación superior'], dtype=object)

In [670]:
df_nivel_educativo_municipios_pequenios['Nivel de estudios'].unique()

array(['Total', 'Educación primaria e inferior',
       'Primera etapa de educación secundaria y similar',
       'Segunda etapa de educación secundaria y educación postsecundaria no superior',
       'Educación superior'], dtype=object)

En el dataframe de nivel de estudios para muncipios pequeños vamos a añadir la columna "País de nacimiento" con el valor "Desconocido"

In [671]:
# Creamos una columna 'País de nacimiento' en el DataFrame de municipios pequeños
df_nivel_educativo_municipios_pequenios['País de nacimiento'] = "Desconocido"

In [672]:
df_nivel_educativo_municipios_pequenios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102570 entries, 0 to 102569
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Municipios          102570 non-null  object
 1   Sexo                102570 non-null  object
 2   Nivel de estudios   102570 non-null  object
 3   Periodo             102570 non-null  int64 
 4   Total               102570 non-null  int64 
 5   País de nacimiento  102570 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.7+ MB


In [673]:
df_nivel_educativo_municipios_grandes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820512 entries, 0 to 820511
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Municipios          820512 non-null  object
 1   Sexo                820512 non-null  object
 2   País de nacimiento  820512 non-null  object
 3   Nivel de estudios   820512 non-null  object
 4   Periodo             820512 non-null  int64 
 5   Total               820512 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.6+ MB


Concatenamos ambos dataframes en uno que agrupe a todos los municipios

In [674]:
# Concatenar los DataFrames
df_nivel_educativo_municipios = pd.concat([df_nivel_educativo_municipios_grandes, df_nivel_educativo_municipios_pequenios], ignore_index=True)

In [675]:
df_nivel_educativo_municipios.sample(20)

,Municipios,Sexo,País de nacimiento,Nivel de estudios,Periodo,Total
306650,18088 Guadahortuna,Mujeres,Total,Educación superior,2022,390
414273,25053 Benavent de Segrià,Total,España,Educación superior,2021,189
548866,33003 Amieva,Total,Total,Segunda etapa de educación secundaria y educac...,2022,420
702437,45082 Lagartera,Hombres,España,Educación superior,2021,10
904079,45010 Aldeanueva de San Bartolomé,Hombres,Desconocido,Segunda etapa de educación secundaria y educac...,2021,25
455218,28026 Brunete,Total,Total,Educación superior,2022,1091
570431,35010 Haría,Mujeres,España,Educación superior,2021,2830
548210,32091 Vilardevós,Mujeres,Total,Educación superior,2022,200
655739,42004 Ágreda,Mujeres,Extranjera,Primera etapa de educación secundaria y similar,2021,450
635273,41003 Albaida del Aljarafe,Hombres,Extranjera,Total,2021,390


Los datos recogidos en el datafrmae son del año 2021 y 2022. Nos quedamos solos con los de 2022 que es el año objeto de nuestro estudo.

In [676]:
# Filtrar el DataFrame para conservar solo las filas donde "Periodo" es 2022
df_nivel_educativo_municipios = df_nivel_educativo_municipios[df_nivel_educativo_municipios['Periodo'] == 2022]
df_nivel_educativo_municipios.reset_index(drop=True, inplace=True)

In [677]:
df_nivel_educativo_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461541 entries, 0 to 461540
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Municipios          461541 non-null  object
 1   Sexo                461541 non-null  object
 2   País de nacimiento  461541 non-null  object
 3   Nivel de estudios   461541 non-null  object
 4   Periodo             461541 non-null  int64 
 5   Total               461541 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 21.1+ MB


In [678]:
df_nivel_educativo_municipios.sample(20)

,Municipios,Sexo,País de nacimiento,Nivel de estudios,Periodo,Total
289651,"36023 Guarda, A",Mujeres,Total,Educación superior,2022,300
234015,28107 Patones,Mujeres,Extranjera,Educación primaria e inferior,2022,2
100210,12053 Chilches/Xilxes,Total,España,Total,2022,2074
55296,08054 Castellbisbal,Hombres,Extranjera,Educación superior,2022,4
213889,25242 Verdú,Hombres,Extranjera,Segunda etapa de educación secundaria y educac...,2022,3
117376,"14057 Rambla, La",Hombres,España,Educación superior,2022,2690
15657,03091 Murla,Total,Extranjera,Segunda etapa de educación secundaria y educac...,2022,15
347920,45032 Camarenilla,Hombres,Total,Educación primaria e inferior,2022,180
191134,23076 Santa Elena,Hombres,España,Educación superior,2022,20
294633,37069 Calvarrasa de Abajo,Total,Total,Educación superior,2022,240


Vamos a pivotar en función de sexo, país de nacimiento y nivel de estudios. Nuestro objetivo es tener toda la información de cada municipio en una fila del DataFrame. Dejamos fuera la columna Periodo, pues al ser todos los datos del 2022 no aporta información relevante.

In [679]:
# Crear Dataframes pivotado por sexo, país de nacimiento y nivel de estudios
df_pivot_sexo = df_nivel_educativo_municipios.pivot_table(
    index='Municipios', 
    columns='Sexo', 
    values='Total', 
    aggfunc='sum'
).reset_index()

df_pivot_pais = df_nivel_educativo_municipios.pivot_table(
    index='Municipios', 
    columns='País de nacimiento', 
    values='Total', 
    aggfunc='sum'
).reset_index()

df_pivot_nivel_estudios = df_nivel_educativo_municipios.pivot_table(
    index='Municipios', 
    columns='Nivel de estudios', 
    values='Total', 
    aggfunc='sum'
).reset_index()



In [680]:
df_pivot_sexo.head()

Sexo,Municipios,Hombres,Mujeres,Total
0,01001 Alegría-Dulantzi,4984,4728,9712
1,01002 Amurrio,17064,17692,34756
2,01003 Aramaio,2572,2400,4972
3,01004 Artziniega,2952,3104,6056
4,01006 Armiñón,206,204,410


In [681]:
df_pivot_pais.head()

País de nacimiento,Municipios,Desconocido,España,Extranjera,Total
0,01001 Alegría-Dulantzi,NaN,8060.0,1652.0,9712.0
1,01002 Amurrio,NaN,31376.0,3380.0,34756.0
2,01003 Aramaio,NaN,4760.0,212.0,4972.0
3,01004 Artziniega,NaN,5596.0,460.0,6056.0
4,01006 Armiñón,820.0,NaN,NaN,NaN


In [682]:
df_pivot_nivel_estudios.head()

Nivel de estudios,Municipios,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,01001 Alegría-Dulantzi,1108,4132,2468,2004,9712
1,01002 Amurrio,4972,13608,7660,8516,34756
2,01003 Aramaio,552,2544,1008,868,4972
3,01004 Artziniega,732,2572,1356,1396,6056
4,01006 Armiñón,36,156,110,108,410


In [683]:
# Combinar las tablas pivote en un único DataFrame
df_nivel_educativo_municipios_transformado = (
    df_pivot_sexo
    .merge(df_pivot_pais, on='Municipios', how='outer')
    .merge(df_pivot_nivel_estudios, on='Municipios', how='outer')
)

# Mostrar las primeras filas del DataFrame transformado
df_nivel_educativo_municipios_transformado.head()

,Municipios,Hombres,Mujeres,Total_x,Desconocido,España,Extranjera,Total_y,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,01001 Alegría-Dulantzi,4984,4728,9712,NaN,8060.0,1652.0,9712.0,1108,4132,2468,2004,9712
1,01002 Amurrio,17064,17692,34756,NaN,31376.0,3380.0,34756.0,4972,13608,7660,8516,34756
2,01003 Aramaio,2572,2400,4972,NaN,4760.0,212.0,4972.0,552,2544,1008,868,4972
3,01004 Artziniega,2952,3104,6056,NaN,5596.0,460.0,6056.0,732,2572,1356,1396,6056
4,01006 Armiñón,206,204,410,820.0,NaN,NaN,NaN,36,156,110,108,410


Vemos que al pivotar se han generado nulos, estos nulos vienen de que para los municipios en el que la información del lugar de nacimiento de los habitantes es desconocida se generan nulos en las columnas "España" y "Extranjera" y viceversa, si se conoce la información de nacimiento se generan nulos para la columna Desconocido. Imputamos por 0 los nulos.

In [684]:
# Imputar valores nulos con 0
df_nivel_educativo_municipios_transformado.fillna(int(0), inplace=True)


In [685]:
df_nivel_educativo_municipios_transformado.head()

,Municipios,Hombres,Mujeres,Total_x,Desconocido,España,Extranjera,Total_y,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,01001 Alegría-Dulantzi,4984,4728,9712,0.0,8060.0,1652.0,9712.0,1108,4132,2468,2004,9712
1,01002 Amurrio,17064,17692,34756,0.0,31376.0,3380.0,34756.0,4972,13608,7660,8516,34756
2,01003 Aramaio,2572,2400,4972,0.0,4760.0,212.0,4972.0,552,2544,1008,868,4972
3,01004 Artziniega,2952,3104,6056,0.0,5596.0,460.0,6056.0,732,2572,1356,1396,6056
4,01006 Armiñón,206,204,410,820.0,0.0,0.0,0.0,36,156,110,108,410


In [686]:
df_nivel_educativo_municipios_transformado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7563 entries, 0 to 7562
Data columns (total 13 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   Municipios                                                                    7563 non-null   object 
 1   Hombres                                                                       7563 non-null   int64  
 2   Mujeres                                                                       7563 non-null   int64  
 3   Total_x                                                                       7563 non-null   int64  
 4   Desconocido                                                                   7563 non-null   float64
 5   España                                                                        7563 non-null   float64
 6   Extranjera                      

Eliminamos las columnas "Total_X" y "Total_y" del Dataframe pues son duplicados de la columa "Total"

In [687]:
# Eliminar las columnas Total_x y Total_y del DataFrame
df_nivel_educativo_municipios_transformado = df_nivel_educativo_municipios_transformado.drop(columns=['Total_x', 'Total_y'])

df_nivel_educativo_municipios_transformado.head()


,Municipios,Hombres,Mujeres,Desconocido,España,Extranjera,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,01001 Alegría-Dulantzi,4984,4728,0.0,8060.0,1652.0,1108,4132,2468,2004,9712
1,01002 Amurrio,17064,17692,0.0,31376.0,3380.0,4972,13608,7660,8516,34756
2,01003 Aramaio,2572,2400,0.0,4760.0,212.0,552,2544,1008,868,4972
3,01004 Artziniega,2952,3104,0.0,5596.0,460.0,732,2572,1356,1396,6056
4,01006 Armiñón,206,204,820.0,0.0,0.0,36,156,110,108,410


In [688]:
# Eliminar las columnas Desconocido, España y Extranjera del DataFrame
df_nivel_educativo_municipios_transformado = df_nivel_educativo_municipios_transformado.drop(columns=['Desconocido', 'España', 'Extranjera'])

# Verificar las columnas restantes
df_nivel_educativo_municipios_transformado.head()


,Municipios,Hombres,Mujeres,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,01001 Alegría-Dulantzi,4984,4728,1108,4132,2468,2004,9712
1,01002 Amurrio,17064,17692,4972,13608,7660,8516,34756
2,01003 Aramaio,2572,2400,552,2544,1008,868,4972
3,01004 Artziniega,2952,3104,732,2572,1356,1396,6056
4,01006 Armiñón,206,204,36,156,110,108,410


Para una lectura mas clara vamos a crear encabezados para cada tipo de información recogida en el Dataframe

In [689]:

# Usamos la función pd.MultiIndex.from_tuples() para crear encabezados y subencabezados
new_columns = pd.MultiIndex.from_tuples(
    [
        ("Municipio", "Nombre del municipio"),
        ("Sexo", "Hombres"),
        ("Sexo", "Mujeres"),
        ("Máximo nivel educativo alcanzado", "Educación primaria e inferior"),
        ("Máximo nivel educativo alcanzado", "Educación superior"),
        ("Máximo nivel educativo alcanzado", "Primera etapa de educación secundaria y similar"),
        ("Máximo nivel educativo alcanzado", "Segunda etapa de educación secundaria y educación postsecundaria no superior"),
        ("Población mayor de 15 años", "Total"),
    ]
)

# Asignar el nuevo MultiIndex a las columnas del DataFrame
df_nivel_educativo_municipios_transformado.columns = new_columns

# Mostrar el DataFrame actualizado
df_nivel_educativo_municipios_transformado.head()


Municipio    Sexo         Máximo nivel educativo alcanzado  \
     Nombre del municipio Hombres Mujeres    Educación primaria e inferior   
0  01001 Alegría-Dulantzi    4984    4728                             1108   
1           01002 Amurrio   17064   17692                             4972   
2           01003 Aramaio    2572    2400                              552   
3        01004 Artziniega    2952    3104                              732   
4           01006 Armiñón     206     204                               36   

                                                                      \
  Educación superior Primera etapa de educación secundaria y similar   
0               4132                                            2468   
1              13608                                            7660   
2               2544                                            1008   
3               2572                                            1356   
4                156                                             110   

                                                                                \
  Segunda etapa de educación secundaria y educación postsecundaria no superior   
0                                               2004                             
1                                               8516                             
2                                                868                             
3                                               1396                             
4                                                108                             

  Población mayor de 15 años  
                       Total  
0                       9712  
1                      34756  
2                       4972  
3                       6056  
4                        410

Reordenamos las columnas de manera que las etapas educativas estenordenadas de "menor a mayor".

In [690]:
# Reordenar las subcolumnas en "Máximo nivel educativo alcanzado"
new_order = [
    ("Municipio", "Nombre del municipio"),
    ("Sexo", "Hombres"),
    ("Sexo", "Mujeres"),
    ("Máximo nivel educativo alcanzado", "Educación primaria e inferior"),
    ("Máximo nivel educativo alcanzado", "Primera etapa de educación secundaria y similar"),
    ("Máximo nivel educativo alcanzado", "Segunda etapa de educación secundaria y educación postsecundaria no superior"),
    ("Máximo nivel educativo alcanzado", "Educación superior"),
    ("Población mayor de 15 años", "Total"),
]

# Reordenar las columnas
df_nivel_educativo_municipios_transformado = df_nivel_educativo_municipios_transformado.reorder_levels([0, 1], axis=1)
df_nivel_educativo_municipios_transformado = df_nivel_educativo_municipios_transformado[new_order]

# Mostrar el DataFrame actualizado con las columnas reordenadas
df_nivel_educativo_municipios_transformado.head()


Municipio    Sexo         Máximo nivel educativo alcanzado  \
     Nombre del municipio Hombres Mujeres    Educación primaria e inferior   
0  01001 Alegría-Dulantzi    4984    4728                             1108   
1           01002 Amurrio   17064   17692                             4972   
2           01003 Aramaio    2572    2400                              552   
3        01004 Artziniega    2952    3104                              732   
4           01006 Armiñón     206     204                               36   

                                                   \
  Primera etapa de educación secundaria y similar   
0                                            2468   
1                                            7660   
2                                            1008   
3                                            1356   
4                                             110   

                                                                                \
  Segunda etapa de educación secundaria y educación postsecundaria no superior   
0                                               2004                             
1                                               8516                             
2                                                868                             
3                                               1396                             
4                                                108                             

                     Población mayor de 15 años  
  Educación superior                      Total  
0               4132                       9712  
1              13608                      34756  
2               2544                       4972  
3               2572                       6056  
4                156                        410

In [691]:
# Dividir la columna "Nombre del municipio" en dos: "Código" y "Nombre del Municipio"
df_nivel_educativo_municipios_transformado[("Municipio", "Código")] = df_nivel_educativo_municipios_transformado[("Municipio", "Nombre del municipio")].apply(
    lambda x: x.split()[0] if isinstance(x, str) else None
)
df_nivel_educativo_municipios_transformado[("Municipio", "Nombre del municipio")] = df_nivel_educativo_municipios_transformado[("Municipio", "Nombre del municipio")].apply(
    lambda x: " ".join(x.split()[1:]) if isinstance(x, str) else None
)

# Reordenar las columnas para que "Código" aparezca antes de "Nombre del municipio"
df_nivel_educativo_municipios_transformado = df_nivel_educativo_municipios_transformado.reorder_levels([0, 1], axis=1)
new_order = [
    ("Municipio", "Código"),
    ("Municipio", "Nombre del municipio"),
    ("Sexo", "Hombres"),
    ("Sexo", "Mujeres"),
    ("Máximo nivel educativo alcanzado", "Educación primaria e inferior"),
    ("Máximo nivel educativo alcanzado", "Primera etapa de educación secundaria y similar"),
    ("Máximo nivel educativo alcanzado", "Segunda etapa de educación secundaria y educación postsecundaria no superior"),
    ("Máximo nivel educativo alcanzado", "Educación superior"),
    ("Población mayor de 15 años", "Total"),
]
df_nivel_educativo_municipios_transformado = df_nivel_educativo_municipios_transformado[new_order]

# Mostrar el DataFrame actualizado
df_nivel_educativo_municipios_transformado.head()


Municipio                         Sexo          \
     Código Nombre del municipio Hombres Mujeres   
0     01001     Alegría-Dulantzi    4984    4728   
1     01002              Amurrio   17064   17692   
2     01003              Aramaio    2572    2400   
3     01004           Artziniega    2952    3104   
4     01006              Armiñón     206     204   

  Máximo nivel educativo alcanzado  \
     Educación primaria e inferior   
0                             1108   
1                             4972   
2                              552   
3                              732   
4                               36   

                                                   \
  Primera etapa de educación secundaria y similar   
0                                            2468   
1                                            7660   
2                                            1008   
3                                            1356   
4                                             110   

                                                                                \
  Segunda etapa de educación secundaria y educación postsecundaria no superior   
0                                               2004                             
1                                               8516                             
2                                                868                             
3                                               1396                             
4                                                108                             

                     Población mayor de 15 años  
  Educación superior                      Total  
0               4132                       9712  
1              13608                      34756  
2               2544                       4972  
3               2572                       6056  
4                156                        410

In [692]:
# Convertir la columna "Código" a tipo Int64
df_nivel_educativo_municipios_transformado[("Municipio", "Código")] = df_nivel_educativo_municipios_transformado[("Municipio", "Código")].astype('int64')

df_nivel_educativo_municipios_transformado.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7563 entries, 0 to 7562
Data columns (total 9 columns):
 #   Column                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                            --------------  ----- 
 0   (Municipio, Código)                                                                                               7563 non-null   int64 
 1   (Municipio, Nombre del municipio)                                                                                 7563 non-null   object
 2   (Sexo, Hombres)                                                                                                   7563 non-null   int64 
 3   (Sexo, Mujeres)                                                                                                   7563 non-null   int64 
 4   (Máximo nivel educativo alcanzado, Ed

Hemos acabado el proceso de limpieza del archivo

In [693]:
df_nivel_educativo_municipios_transformado.to_csv("datos_limpiados/municipios/nivel_educativo_municipios_limpio.csv", index=False)

## Procesamos ahora el archivo que contiene la información provincial, autonómica y nacional del nivel educativo alcanzado.

In [694]:
df_nivel_educativo = pd.read_csv("datos/ine/nivel_educativo.csv", encoding= 'latin-1', sep=';')

In [695]:
df_nivel_educativo.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Lugar de nacimiento,Nivel de formación alcanzado,Edad,Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,España,Doctorado universitario,15 y más años,Hombres,2022,197.035
1,Total Nacional,NaN,NaN,España,Doctorado universitario,15 y más años,Mujeres,2022,168.651
2,Total Nacional,NaN,NaN,España,Doctorado universitario,15 y más años,Total,2022,365.686
3,Total Nacional,NaN,NaN,España,Educación primaria,15 y más años,Hombres,2022,1.500.259
4,Total Nacional,NaN,NaN,España,Educación primaria,15 y más años,Mujeres,2022,2.120.412


Eliminamos la columna "Edad" que no es relevante para nuestro análisis

In [696]:
# Eliminar las columnas 'Edad'
df_nivel_educativo.drop(columns=['Edad'], inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_nivel_educativo.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Lugar de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,España,Doctorado universitario,Hombres,2022,197.035
1,Total Nacional,NaN,NaN,España,Doctorado universitario,Mujeres,2022,168.651
2,Total Nacional,NaN,NaN,España,Doctorado universitario,Total,2022,365.686
3,Total Nacional,NaN,NaN,España,Educación primaria,Hombres,2022,1.500.259
4,Total Nacional,NaN,NaN,España,Educación primaria,Mujeres,2022,2.120.412


Formateamos la columna "Total"

In [697]:
# Convertimos la columna "Total" a int64
df_nivel_educativo['Total'] = df_nivel_educativo['Total'].str.replace('.', '').astype('int64')
df_nivel_educativo.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Lugar de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,España,Doctorado universitario,Hombres,2022,197035
1,Total Nacional,NaN,NaN,España,Doctorado universitario,Mujeres,2022,168651
2,Total Nacional,NaN,NaN,España,Doctorado universitario,Total,2022,365686
3,Total Nacional,NaN,NaN,España,Educación primaria,Hombres,2022,1500259
4,Total Nacional,NaN,NaN,España,Educación primaria,Mujeres,2022,2120412


A partir del dataframe que estamos procesando vamos a crear 3, uno con la información nacional, otro con la información autonomica y otro con la información provincial.

In [698]:
# Filtrar el DataFrame para obtener los tres niveles de información

# Nivel nacional: Filtramos donde "Comunidades y Ciudades Autónomas" sea NaN
df_nivel_educativo_nacional_filtrado = df_nivel_educativo[df_nivel_educativo['Comunidades y Ciudades Autónomas'].isna()].copy().reset_index(drop=True)

# Nivel autonómico: Filtramos donde "Comunidades y Ciudades Autónomas" no sea NaN y "Provincias" sea NaN
df_nivel_educativo_autonomico_filtrado = df_nivel_educativo[df_nivel_educativo['Comunidades y Ciudades Autónomas'].notna() & df_nivel_educativo['Provincias'].isna()].copy().reset_index(drop=True)

# Nivel provincial: Filtramos donde "Provincias" no sea NaN
df_nivel_educativo_provincial_filtrado = df_nivel_educativo[df_nivel_educativo['Provincias'].notna()].copy().reset_index(drop=True)



In [699]:
df_nivel_educativo_nacional_filtrado.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Lugar de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,España,Doctorado universitario,Hombres,2022,197035
1,Total Nacional,NaN,NaN,España,Doctorado universitario,Mujeres,2022,168651
2,Total Nacional,NaN,NaN,España,Doctorado universitario,Total,2022,365686
3,Total Nacional,NaN,NaN,España,Educación primaria,Hombres,2022,1500259
4,Total Nacional,NaN,NaN,España,Educación primaria,Mujeres,2022,2120412


In [700]:
df_nivel_educativo_autonomico_filtrado.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Lugar de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,01 Andalucía,NaN,España,Doctorado universitario,Hombres,2022,30241
1,Total Nacional,01 Andalucía,NaN,España,Doctorado universitario,Mujeres,2022,23208
2,Total Nacional,01 Andalucía,NaN,España,Doctorado universitario,Total,2022,53449
3,Total Nacional,01 Andalucía,NaN,España,Educación primaria,Hombres,2022,290619
4,Total Nacional,01 Andalucía,NaN,España,Educación primaria,Mujeres,2022,360800


In [701]:
df_nivel_educativo_provincial_filtrado.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,Lugar de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,01 Andalucía,04 Almería,España,Doctorado universitario,Hombres,2022,1673
1,Total Nacional,01 Andalucía,04 Almería,España,Doctorado universitario,Mujeres,2022,1354
2,Total Nacional,01 Andalucía,04 Almería,España,Doctorado universitario,Total,2022,3027
3,Total Nacional,01 Andalucía,04 Almería,España,Educación primaria,Hombres,2022,28248
4,Total Nacional,01 Andalucía,04 Almería,España,Educación primaria,Mujeres,2022,29773


## Limpiamos ahora  los datos de nivel educativo alcanzado por provincias.

In [702]:
df_nivel_educativo_provincias = pd.read_csv("datos/ine/nivel_educativo_provincias.csv", encoding= 'latin-1', sep=';')

In [703]:
df_nivel_educativo_provincias.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,01 Andalucía,04 Almería,Total,Total,Total,2022,618.592
1,Total Nacional,01 Andalucía,04 Almería,Total,Total,Hombres,2022,315.700
2,Total Nacional,01 Andalucía,04 Almería,Total,Total,Mujeres,2022,302.892
3,Total Nacional,01 Andalucía,04 Almería,Total,Educación primaria e inferior,Total,2022,164.692
4,Total Nacional,01 Andalucía,04 Almería,Total,Educación primaria e inferior,Hombres,2022,86.587


Procesamos el dataframe con la información provincial de nivel de educación

In [704]:
def procesar_informacion_provincias(df):
    """
    Realiza el procesamiento de un Dataframe con información a nivel provincial, extraida del INE,
    aplicando las siguientes transformaciones:
    1. Elimina la columna "Total Nacional".
    2. Separa los códigos y nombres de comunidades y provincias en columnas distintas.
    3. Convierte las columnas 'Código comunidad' y 'Código provincia' a tipo int64.
    4. Reordena las columnas colocando los códigos y nombres de comunidades y provincias al inicio.
        
    Parámetros:
        df (pd.DataFrame): DataFrame con la información de nivel educativo por provincias.
    
    Retorna:
        pd.DataFrame: DataFrame transformado con las modificaciones aplicadas.
    """
    df = df.copy()  # Crear una copia para no modificar el original
    
    # 1. Eliminar la columna "Total Nacional"
    if "Total Nacional" in df.columns:
        df.drop(columns=["Total Nacional"], inplace=True)
           
    # 2. Separar código y nombre de comunidad y provincia
    if 'Comunidades y Ciudades Autónomas' in df.columns and 'Provincias' in df.columns:
        df[['Código comunidad', 'Nombre comunidad']] = df['Comunidades y Ciudades Autónomas'].str.split(pat=' ', n=1, expand=True)
        df[['Código provincia', 'Nombre provincia']] = df['Provincias'].str.split(pat=' ', n=1, expand=True)
        df.drop(columns=['Comunidades y Ciudades Autónomas', 'Provincias'], inplace=True)

    # 3. Convertir las columnas 'Código comunidad' y 'Código provincia' a tipo int64 si existen
    for col in ['Código comunidad', 'Código provincia']:
        if col in df.columns:
            df[col] = df[col].astype('int64')
    
    # 4. Reordenar las columnas para que los códigos y nombres aparezcan primero
    columnas_prioritarias = ['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia']
    columnas_ordenadas = columnas_prioritarias + [col for col in df.columns if col not in columnas_prioritarias]
    df = df[columnas_ordenadas]
        
    return df

# Aplicar la función al DataFrame
df_nivel_educativo_provincias = procesar_informacion_provincias(df_nivel_educativo_provincias)

# Mostrar información y primeras filas del DataFrame actualizado
print(df_nivel_educativo_provincias.info())
df_nivel_educativo_provincias.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16380 entries, 0 to 16379
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Código comunidad              16380 non-null  int64 
 1   Nombre comunidad              16380 non-null  object
 2   Código provincia              16380 non-null  int64 
 3   Nombre provincia              16380 non-null  object
 4   País de nacimiento            16380 non-null  object
 5   Nivel de formación alcanzado  16380 non-null  object
 6   Sexo                          16380 non-null  object
 7   Periodo                       16380 non-null  int64 
 8   Total                         16380 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.1+ MB
None


,Código comunidad,Nombre comunidad,Código provincia,Nombre provincia,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,1,Andalucía,4,Almería,Total,Total,Total,2022,618.592
1,1,Andalucía,4,Almería,Total,Total,Hombres,2022,315.700
2,1,Andalucía,4,Almería,Total,Total,Mujeres,2022,302.892
3,1,Andalucía,4,Almería,Total,Educación primaria e inferior,Total,2022,164.692
4,1,Andalucía,4,Almería,Total,Educación primaria e inferior,Hombres,2022,86.587


In [705]:
# Convertimos la columna "Total" a int64
df_nivel_educativo_provincias['Total'] = df_nivel_educativo_provincias['Total'].str.replace('.', '').astype('int64')

# Mostramos el DataFrame actualizado
df_nivel_educativo_provincias.head()

,Código comunidad,Nombre comunidad,Código provincia,Nombre provincia,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,1,Andalucía,4,Almería,Total,Total,Total,2022,618592
1,1,Andalucía,4,Almería,Total,Total,Hombres,2022,315700
2,1,Andalucía,4,Almería,Total,Total,Mujeres,2022,302892
3,1,Andalucía,4,Almería,Total,Educación primaria e inferior,Total,2022,164692
4,1,Andalucía,4,Almería,Total,Educación primaria e inferior,Hombres,2022,86587


Ceuta y Melilla son ciudades autónomas tenemos que decidir que hacer con ellas

In [706]:
"""
# Filtrar el DataFrame eliminando las filas donde el Nombre provincia sea "Ceuta" o "Melilla"
df_nivel_educativo_provincias = df_nivel_educativo_provincias[
    ~df_nivel_educativo_provincias["Nombre provincia"].isin(["Ceuta", "Melilla"])
]

# Redf_nivel_educativo_provinciassetear el índice después de eliminar las filas no deseadas
df_censo_2022_provincial = df_nivel_educativo_provincias.reset_index(drop=True)
"""

'\n# Filtrar el DataFrame eliminando las filas donde el Nombre provincia sea "Ceuta" o "Melilla"\ndf_nivel_educativo_provincias = df_nivel_educativo_provincias[\n    ~df_nivel_educativo_provincias["Nombre provincia"].isin(["Ceuta", "Melilla"])\n]\n\n# Redf_nivel_educativo_provinciassetear el índice después de eliminar las filas no deseadas\ndf_censo_2022_provincial = df_nivel_educativo_provincias.reset_index(drop=True)\n'

Vamos a pivotar columnas del dataframe para que cada fila contenga toda la información de una provincia. Generamos encabezados dobles que agrupen la información por categorías.

In [707]:
def pivotar_y_generar_encabezados_provincias(df):
    """
    Transforma un DataFrame mediante la pivotación de tres variables clave 
    ('País de nacimiento', 'Nivel de formación alcanzado' y 'Sexo') 
    y reorganiza los encabezados con un índice de múltiples niveles.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame que contiene las columnas:
        - 'Código comunidad'
        - 'Nombre comunidad'
        - 'Código provincia'
        - 'Nombre provincia'
        - 'País de nacimiento'
        - 'Nivel de formación alcanzado'
        - 'Sexo'
        - 'Total'
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame transformado con las columnas pivotadas y organizadas bajo un 
        MultiIndex donde las categorías corresponden a la variable a la que pertenecen.
        Se incluyen los siguientes niveles de encabezado:
        - 'Comunidades y ciudades autónomas' (Código y Nombre de comunidad)
        - 'Provincias' (Código y Nombre de provincia)
        - 'Sexo' (Hombres y Mujeres)
        - 'País de nacimiento' (Cada país representado en la data)
        - 'Nivel educativo' (Diferentes niveles de formación)
        - 'Población mayor de 15 años' (Total)
    """

    # 1. Verificación de columnas necesarias
    columnas_necesarias = [
        'Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia', 
        'País de nacimiento', 'Nivel de formación alcanzado', 'Sexo', 'Total'
    ]
    
    for col in columnas_necesarias:
        if col not in df.columns:
            raise ValueError(f"Falta la columna necesaria en el DataFrame: {col}")

    # 2. Pivotamos la columna "País de nacimiento"
    if 'País de nacimiento' in df.columns:
        df_pivotado_pais = df.pivot_table(
            index=['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia'],
            columns='País de nacimiento',
            values='Total',
            aggfunc='sum'
        ).reset_index()
    else:
        df_pivotado_pais = pd.DataFrame()

    # 3. Pivotamos la columna "Nivel de formación alcanzado"
    if 'Nivel de formación alcanzado' in df.columns:
        df_pivotado_nivel = df.pivot_table(
            index=['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia'],
            columns='Nivel de formación alcanzado',
            values='Total',
            aggfunc='sum'
        ).reset_index()
    else:
        df_pivotado_nivel = pd.DataFrame()

    # 4. Pivotamos la columna "Sexo"
    if 'Sexo' in df.columns:
        df_pivotado_sexo = df.pivot_table(
            index=['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia'],
            columns='Sexo',
            values='Total',
            aggfunc='sum'
        ).reset_index()
    else:
        df_pivotado_sexo = pd.DataFrame()

    # 5. Fusionamos los DataFrames resultantes con outer join
    df_concatenado = df_pivotado_pais.copy() if not df_pivotado_pais.empty else pd.DataFrame()
    
    if not df_pivotado_nivel.empty:
        df_concatenado = pd.merge(
            df_concatenado, df_pivotado_nivel, 
            on=['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia'], 
            how='outer'
        ) if not df_concatenado.empty else df_pivotado_nivel
    
    if not df_pivotado_sexo.empty:
        df_concatenado = pd.merge(
            df_concatenado, df_pivotado_sexo, 
            on=['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia'], 
            how='outer'
        ) if not df_concatenado.empty else df_pivotado_sexo
    
    # 6. Eliminamos columnas duplicadas si existen
    columnas_duplicadas = [col for col in ['Total_x', 'Total_y'] if col in df_concatenado.columns]
    if columnas_duplicadas:
        df_concatenado = df_concatenado.drop(columns=columnas_duplicadas)

    # 7. Reordenamos las columnas para ubicar 'Hombres' y 'Mujeres' después de 'Nombre provincia'
    columnas_base = ['Código comunidad', 'Nombre comunidad', 'Código provincia', 'Nombre provincia']
    columnas_sexo = ['Hombres', 'Mujeres'] if 'Hombres' in df_concatenado.columns and 'Mujeres' in df_concatenado.columns else []
    
    otras_columnas = [col for col in df_concatenado.columns if col not in columnas_base + columnas_sexo]
    df_concatenado = df_concatenado[columnas_base + columnas_sexo + otras_columnas]

    # 8. Creamos una lista de categorías para las columnas del DataFrame
    categorias = (
        ['Comunidades y ciudades autónomas'] * 2 +  # Código comunidad, Nombre comunidad
        ['Provincias'] * 2 +  # Código provincia, Nombre provincia
        (['Sexo'] * 2 if 'Hombres' in df_concatenado.columns and 'Mujeres' in df_concatenado.columns else []) +  # Sexo
        ['País de nacimiento'] * (len(df_pivotado_pais.columns) - 4 if not df_pivotado_pais.empty else 0) +  # Países de nacimiento
        ['Nivel educativo'] * (len(df_pivotado_nivel.columns) - 4 if not df_pivotado_nivel.empty else 0) +  # Niveles educativos
        ['Población mayor de 15 años'] * (1 if 'Total' in df_concatenado.columns else 0)  # Total
    )

    # 9. Asignamos las categorías a las columnas del DataFrame
    if len(categorias) == len(df_concatenado.columns):
        df_concatenado.columns = pd.MultiIndex.from_tuples(zip(categorias, df_concatenado.columns))
    
    return df_concatenado



# Aplicar la función al DataFrame
df_nivel_educativo_provincias = pivotar_y_generar_encabezados_provincias(df_nivel_educativo_provincias)

# Mostrar las primeras filas y la información del DataFrame actualizado
df_nivel_educativo_provincias.head()
# df_nivel_educativo_provincias.info()


,Código comunidad,Nombre comunidad,Código provincia,Nombre provincia,Hombres,Mujeres,Argentina,Bolivia,China,Colombia,Cuba,Ecuador,España,Francia,Marruecos,Otros países de América,Otros países de Asia,Otros países de Europa,Otros países de Oceanía,Otros países de África,Perú,Reino Unido,República Dominicana,Rumanía,Ucrania,Venezuela,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,1,Andalucía,4,Almería,1262800,1211568,22836,4880,3784,21328,3112,27780,1837208,13496,205044,16976,8260,66520,312,73992,11324,65544,3908,75224,5916,6924,658768,568804,818604,428192,2474368
1,1,Andalucía,11,Cádiz,2088800,2177956,10892,9744,7628,13840,4848,3696,3980088,12560,64768,30644,7884,49552,776,12124,5168,28136,5000,8716,2804,7888,743224,1120036,1516532,886964,4266756
2,1,Andalucía,14,Córdoba,1298348,1374124,2796,1972,3416,8908,2056,7960,2549124,5156,15224,18616,3476,17148,296,6112,1960,4348,1496,16380,1820,4208,485528,737780,944780,504384,2672472
3,1,Andalucía,18,Granada,1554956,1619956,20732,16844,7168,18380,4260,9396,2819328,16636,73360,29676,8372,54332,572,20576,4088,25636,4232,28420,2416,10488,525080,976448,1064516,608868,3174912
4,1,Andalucía,21,Huelva,892656,920400,2152,1816,2616,8456,1968,4244,1586476,2540,64108,9628,1980,38248,92,21540,1680,4536,1372,52520,3512,3572,342592,444416,714704,311344,1813056


In [708]:
# Guardamos el DataFrame en un archivo CSV
df_nivel_educativo_provincias.to_csv("datos_limpiados/provincias/nivel_educativo_provincias_limpio.csv", index=False)

## Limpiamos ahora los datos de nivel educativo alcanzado por comunidades autónomas

In [709]:
df_nivel_educativo_comunidades = pd.read_csv("datos/ine/nivel_educativo_comunidades.csv", encoding= 'latin-1', sep=';')

In [710]:
df_nivel_educativo_comunidades.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,01 Andalucía,NaN,Total,Total,Total,2022,7.251.427
1,Total Nacional,01 Andalucía,NaN,Total,Total,Hombres,2022,3.547.880
2,Total Nacional,01 Andalucía,NaN,Total,Total,Mujeres,2022,3.703.547
3,Total Nacional,01 Andalucía,NaN,Total,Educación primaria e inferior,Total,2022,1.332.260
4,Total Nacional,01 Andalucía,NaN,Total,Educación primaria e inferior,Hombres,2022,583.687


La columna 'Provincias' está vacía así que la eliminaos

In [711]:
# Eliminar la columna 'Provincias'
df_nivel_educativo_comunidades.drop(columns=['Provincias'], inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_nivel_educativo_comunidades.head()


,Total Nacional,Comunidades y Ciudades Autónomas,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,01 Andalucía,Total,Total,Total,2022,7.251.427
1,Total Nacional,01 Andalucía,Total,Total,Hombres,2022,3.547.880
2,Total Nacional,01 Andalucía,Total,Total,Mujeres,2022,3.703.547
3,Total Nacional,01 Andalucía,Total,Educación primaria e inferior,Total,2022,1.332.260
4,Total Nacional,01 Andalucía,Total,Educación primaria e inferior,Hombres,2022,583.687


Procesamos el Dataframe con la información autonómica.

In [712]:
def procesar_informacion_comunidades(df):
    """
    Realiza el procesamiento de un Dataframe con información a nivel de comunidad autónoma, extraida del INE,
    aplicando las siguientes transformaciones:
    1. Elimina la columna "Total Nacional".
    2. Separa los códigos y nombres de comunidades en columnas distintas.
    3. Convierte la columna 'Código comunidad' a tipo int64.
    4. Reordena las columnas colocando los códigos y nombres de comunidades al inicio.
        
    Parámetros:
        df (pd.DataFrame): DataFrame con la información de nivel educativo por provincias.
    
    Retorna:
        pd.DataFrame: DataFrame transformado con las modificaciones aplicadas.
    """
    df = df.copy()  # Crear una copia para no modificar el original
    
    # 1. Eliminar la columna "Total Nacional"
    if "Total Nacional" in df.columns:
        df.drop(columns=["Total Nacional"], inplace=True)
           
    # 2. Separar código y nombre de comunidad
    if 'Comunidades y Ciudades Autónomas' in df.columns:
        df[['Código comunidad', 'Nombre comunidad']] = df['Comunidades y Ciudades Autónomas'].str.split(pat=' ', n=1, expand=True)
        df.drop(columns=['Comunidades y Ciudades Autónomas'], inplace=True)

    # 3. Convertir la columnas 'Código comunidad' a tipo int64 si existe
    for col in ['Código comunidad']:
        if col in df.columns:
            df[col] = df[col].astype('int64')
    
    # 4. Reordenar las columnas para que el codigo y nombre de comunidad aparezcan primero
    columnas_prioritarias = ['Código comunidad', 'Nombre comunidad']
    columnas_ordenadas = columnas_prioritarias + [col for col in df.columns if col not in columnas_prioritarias]
    df = df[columnas_ordenadas]
        
    return df

# Aplicar la función al DataFrame
df_nivel_educativo_comunidades = procesar_informacion_comunidades(df_nivel_educativo_comunidades)

# Mostrar información y primeras filas del DataFrame actualizado
print(df_nivel_educativo_comunidades.info())
df_nivel_educativo_comunidades.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5985 entries, 0 to 5984
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Código comunidad              5985 non-null   int64 
 1   Nombre comunidad              5985 non-null   object
 2   País de nacimiento            5985 non-null   object
 3   Nivel de formación alcanzado  5985 non-null   object
 4   Sexo                          5985 non-null   object
 5   Periodo                       5985 non-null   int64 
 6   Total                         5985 non-null   object
dtypes: int64(2), object(5)
memory usage: 327.4+ KB
None


,Código comunidad,Nombre comunidad,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,1,Andalucía,Total,Total,Total,2022,7.251.427
1,1,Andalucía,Total,Total,Hombres,2022,3.547.880
2,1,Andalucía,Total,Total,Mujeres,2022,3.703.547
3,1,Andalucía,Total,Educación primaria e inferior,Total,2022,1.332.260
4,1,Andalucía,Total,Educación primaria e inferior,Hombres,2022,583.687


Formateamos la columna "Total" 

In [713]:
# Convertimos la columna "Total" a int64 eliminando puntos de miles
df_nivel_educativo_comunidades['Total'] = df_nivel_educativo_comunidades['Total'].str.replace('.', '').astype('int64')

Hacemos la pivotación bien hecha para Comunidades Autónomas

In [714]:
# Pivotar nuevamente incluyendo "País de nacimiento"
df_pivotado_completo = df_nivel_educativo_comunidades.pivot_table(
    index=['Código comunidad', 'Nombre comunidad', 'Periodo'],
    columns=['País de nacimiento', 'Nivel de formación alcanzado', 'Sexo'],
    values='Total',
    aggfunc='sum'
).reset_index()

# Renombrar columnas para mayor claridad
df_pivotado_completo.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_pivotado_completo.columns]
df_pivotado_completo.rename(columns={'Código comunidad_': 'Código comunidad', 'Nombre comunidad_': 'Nombre comunidad', 'Periodo_': 'Periodo'}, inplace=True)

# Mostrar el DataFrame transformado
df_pivotado_completo.head()


,Código comunidad__,Nombre comunidad__,Periodo__,Argentina_Educación primaria e inferior_Hombres,Argentina_Educación primaria e inferior_Mujeres,Argentina_Educación primaria e inferior_Total,Argentina_Educación superior_Hombres,Argentina_Educación superior_Mujeres,Argentina_Educación superior_Total,Argentina_Primera etapa de educación secundaria y similar_Hombres,Argentina_Primera etapa de educación secundaria y similar_Mujeres,Argentina_Primera etapa de educación secundaria y similar_Total,Argentina_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Argentina_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Argentina_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Argentina_Total_Hombres,Argentina_Total_Mujeres,Argentina_Total_Total,Bolivia_Educación primaria e inferior_Hombres,Bolivia_Educación primaria e inferior_Mujeres,Bolivia_Educación primaria e inferior_Total,Bolivia_Educación superior_Hombres,Bolivia_Educación superior_Mujeres,Bolivia_Educación superior_Total,Bolivia_Primera etapa de educación secundaria y similar_Hombres,Bolivia_Primera etapa de educación secundaria y similar_Mujeres,Bolivia_Primera etapa de educación secundaria y similar_Total,Bolivia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Bolivia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Bolivia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Bolivia_Total_Hombres,Bolivia_Total_Mujeres,Bolivia_Total_Total,China_Educación primaria e inferior_Hombres,China_Educación primaria e inferior_Mujeres,China_Educación primaria e inferior_Total,China_Educación superior_Hombres,China_Educación superior_Mujeres,China_Educación superior_Total,China_Primera etapa de educación secundaria y similar_Hombres,China_Primera etapa de educación secundaria y similar_Mujeres,China_Primera etapa de educación secundaria y similar_Total,China_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,China_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,China_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,China_Total_Hombres,China_Total_Mujeres,China_Total_Total,Colombia_Educación primaria e inferior_Hombres,Colombia_Educación primaria e inferior_Mujeres,Colombia_Educación primaria e inferior_Total,Colombia_Educación superior_Hombres,Colombia_Educación superior_Mujeres,Colombia_Educación superior_Total,Colombia_Primera etapa de educación secundaria y similar_Hombres,Colombia_Primera etapa de educación secundaria y similar_Mujeres,Colombia_Primera etapa de educación secundaria y similar_Total,Colombia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Colombia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Colombia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Colombia_Total_Hombres,Colombia_Total_Mujeres,Colombia_Total_Total,Cuba_Educación primaria e inferior_Hombres,Cuba_Educación primaria e inferior_Mujeres,Cuba_Educación primaria e inferior_Total,Cuba_Educación superior_Hombres,Cuba_Educación superior_Mujeres,Cuba_Educación superior_Total,Cuba_Primera etapa de educación secundaria y similar_Hombres,Cuba_Primera etapa de educación secundaria y similar_Mujeres,Cuba_Primera etapa de educación secundaria y similar_Total,Cuba_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Cuba_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Cuba_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Cuba_Total_Hombres,Cuba_Total_Mujeres,Cuba_Total_Total,Ecuador_Educación primaria e inferior_Hombres,Ecuador_Educación primaria e inferior_Mujeres,Ecuador_Educación primaria e inferior_T

In [715]:
df_nivel_educativo_comunidades.to_csv("datos_limpiados/comunidades/nivel_educativo_comunidades_sin_pivotar.csv", index=False)

In [716]:
# Pivotar el DataFrame para que cada fila represente una comunidad autónoma
df_pivotado= df_nivel_educativo_comunidades.pivot_table(
    index=['Código comunidad', 'Nombre comunidad', 'Periodo'],
    columns=['País de nacimiento', 'Nivel de formación alcanzado', 'Sexo'],
    values='Total',
    aggfunc='sum'
).reset_index()

# Renombrar columnas para mayor claridad
df_pivotado.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_pivotado.columns]

# Mostrar las primeras filas del DataFrame transformado
df_pivotado.head()


,Código comunidad__,Nombre comunidad__,Periodo__,Argentina_Educación primaria e inferior_Hombres,Argentina_Educación primaria e inferior_Mujeres,Argentina_Educación primaria e inferior_Total,Argentina_Educación superior_Hombres,Argentina_Educación superior_Mujeres,Argentina_Educación superior_Total,Argentina_Primera etapa de educación secundaria y similar_Hombres,Argentina_Primera etapa de educación secundaria y similar_Mujeres,Argentina_Primera etapa de educación secundaria y similar_Total,Argentina_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Argentina_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Argentina_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Argentina_Total_Hombres,Argentina_Total_Mujeres,Argentina_Total_Total,Bolivia_Educación primaria e inferior_Hombres,Bolivia_Educación primaria e inferior_Mujeres,Bolivia_Educación primaria e inferior_Total,Bolivia_Educación superior_Hombres,Bolivia_Educación superior_Mujeres,Bolivia_Educación superior_Total,Bolivia_Primera etapa de educación secundaria y similar_Hombres,Bolivia_Primera etapa de educación secundaria y similar_Mujeres,Bolivia_Primera etapa de educación secundaria y similar_Total,Bolivia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Bolivia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Bolivia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Bolivia_Total_Hombres,Bolivia_Total_Mujeres,Bolivia_Total_Total,China_Educación primaria e inferior_Hombres,China_Educación primaria e inferior_Mujeres,China_Educación primaria e inferior_Total,China_Educación superior_Hombres,China_Educación superior_Mujeres,China_Educación superior_Total,China_Primera etapa de educación secundaria y similar_Hombres,China_Primera etapa de educación secundaria y similar_Mujeres,China_Primera etapa de educación secundaria y similar_Total,China_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,China_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,China_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,China_Total_Hombres,China_Total_Mujeres,China_Total_Total,Colombia_Educación primaria e inferior_Hombres,Colombia_Educación primaria e inferior_Mujeres,Colombia_Educación primaria e inferior_Total,Colombia_Educación superior_Hombres,Colombia_Educación superior_Mujeres,Colombia_Educación superior_Total,Colombia_Primera etapa de educación secundaria y similar_Hombres,Colombia_Primera etapa de educación secundaria y similar_Mujeres,Colombia_Primera etapa de educación secundaria y similar_Total,Colombia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Colombia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Colombia_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Colombia_Total_Hombres,Colombia_Total_Mujeres,Colombia_Total_Total,Cuba_Educación primaria e inferior_Hombres,Cuba_Educación primaria e inferior_Mujeres,Cuba_Educación primaria e inferior_Total,Cuba_Educación superior_Hombres,Cuba_Educación superior_Mujeres,Cuba_Educación superior_Total,Cuba_Primera etapa de educación secundaria y similar_Hombres,Cuba_Primera etapa de educación secundaria y similar_Mujeres,Cuba_Primera etapa de educación secundaria y similar_Total,Cuba_Segunda etapa de educación secundaria y educación postsecundaria no superior_Hombres,Cuba_Segunda etapa de educación secundaria y educación postsecundaria no superior_Mujeres,Cuba_Segunda etapa de educación secundaria y educación postsecundaria no superior_Total,Cuba_Total_Hombres,Cuba_Total_Mujeres,Cuba_Total_Total,Ecuador_Educación primaria e inferior_Hombres,Ecuador_Educación primaria e inferior_Mujeres,Ecuador_Educación primaria e inferior_T

In [717]:
df_pivotado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 318 entries, Código comunidad__ to Venezuela_Total_Total
dtypes: int64(317), object(1)
memory usage: 47.3+ KB


Vamos a pivotar columnas del dataframe para que cada fila contenga toda la información de una Comunidad Autónoma. Generamos encabezados dobles que agrupen la información por categorías.

In [718]:
def pivotar_y_generar_encabezados_comunidades(df):
    """
    Transforma un DataFrame mediante la pivotación de tres variables clave 
    ('País de nacimiento', 'Nivel de formación alcanzado' y 'Sexo') 
    y reorganiza los encabezados con un índice de múltiples niveles.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame que contiene las columnas:
        - 'Código comunidad'
        - 'Nombre comunidad'
        - 'País de nacimiento'
        - 'Nivel de formación alcanzado'
        - 'Sexo'
        - 'Total'
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame transformado con las columnas pivotadas y organizadas bajo un 
        MultiIndex donde las categorías corresponden a la variable a la que pertenecen.
        Se incluyen los siguientes niveles de encabezado:
        - 'Comunidades y ciudades autónomas' (Código y Nombre de comunidad)
        - 'Sexo' (Hombres y Mujeres)
        - 'País de nacimiento' (Cada país representado en la data)
        - 'Nivel educativo' (Diferentes niveles de formación)
        - 'Población mayor de 15 años' (Total)
    """

    # 1. Verificación de columnas necesarias
    columnas_necesarias = [
        'Código comunidad', 'Nombre comunidad', 'País de nacimiento', 'Nivel de formación alcanzado', 'Sexo', 'Total'
    ]
    
    for col in columnas_necesarias:
        if col not in df.columns:
            raise ValueError(f"Falta la columna necesaria en el DataFrame: {col}")

    # 2. Pivotamos la columna "País de nacimiento"
    if 'País de nacimiento' in df.columns:
        df_pivotado_pais = df.pivot_table(
            index=['Código comunidad', 'Nombre comunidad'],
            columns='País de nacimiento',
            values='Total',
            aggfunc='sum'
        ).reset_index()
    else:
        df_pivotado_pais = pd.DataFrame()

    # 3. Pivotamos la columna "Nivel de formación alcanzado"
    if 'Nivel de formación alcanzado' in df.columns:
        df_pivotado_nivel = df.pivot_table(
            index=['Código comunidad', 'Nombre comunidad'],
            columns='Nivel de formación alcanzado',
            values='Total',
            aggfunc='sum'
        ).reset_index()
    else:
        df_pivotado_nivel = pd.DataFrame()

    # 4. Pivotamos la columna "Sexo"
    if 'Sexo' in df.columns:
        df_pivotado_sexo = df.pivot_table(
            index=['Código comunidad', 'Nombre comunidad'],
            columns='Sexo',
            values='Total',
            aggfunc='sum'
        ).reset_index()
    else:
        df_pivotado_sexo = pd.DataFrame()

    # 5. Fusionamos los DataFrames resultantes con outer join
    df_concatenado = df_pivotado_pais.copy() if not df_pivotado_pais.empty else pd.DataFrame()
    
    if not df_pivotado_nivel.empty:
        df_concatenado = pd.merge(
            df_concatenado, df_pivotado_nivel, 
            on=['Código comunidad', 'Nombre comunidad'], 
            how='outer'
        ) if not df_concatenado.empty else df_pivotado_nivel
    
    if not df_pivotado_sexo.empty:
        df_concatenado = pd.merge(
            df_concatenado, df_pivotado_sexo, 
            on=['Código comunidad', 'Nombre comunidad'], 
            how='outer'
        ) if not df_concatenado.empty else df_pivotado_sexo
    
    # 6. Eliminamos columnas duplicadas si existen
    columnas_duplicadas = [col for col in ['Total_x', 'Total_y'] if col in df_concatenado.columns]
    if columnas_duplicadas:
        df_concatenado = df_concatenado.drop(columns=columnas_duplicadas)

    # 7. Reordenamos las columnas para ubicar 'Hombres' y 'Mujeres' después de 'Nombre provincia'
    columnas_base = ['Código comunidad', 'Nombre comunidad']
    columnas_sexo = ['Hombres', 'Mujeres'] if 'Hombres' in df_concatenado.columns and 'Mujeres' in df_concatenado.columns else []
    
    otras_columnas = [col for col in df_concatenado.columns if col not in columnas_base + columnas_sexo]
    df_concatenado = df_concatenado[columnas_base + columnas_sexo + otras_columnas]

    # 8. Creamos una lista de categorías para las columnas del DataFrame
    categorias = (
        ['Comunidades y ciudades autónomas'] * 2 +  # Código comunidad, Nombre comunidad
        (['Sexo'] * 2 if 'Hombres' in df_concatenado.columns and 'Mujeres' in df_concatenado.columns else []) +  # Sexo
        ['País de nacimiento'] * (len(df_pivotado_pais.columns) - 4 if not df_pivotado_pais.empty else 0) +  # Países de nacimiento
        ['Nivel educativo'] * (len(df_pivotado_nivel.columns) - 4 if not df_pivotado_nivel.empty else 0) +  # Niveles educativos
        ['Población mayor de 15 años'] * (1 if 'Total' in df_concatenado.columns else 0)  # Total
    )

    # 9. Asignamos las categorías a las columnas del DataFrame
    if len(categorias) == len(df_concatenado.columns):
        df_concatenado.columns = pd.MultiIndex.from_tuples(zip(categorias, df_concatenado.columns))
    
    return df_concatenado


# Aplicar la función al DataFrame
df_nivel_educativo_comunidades = pivotar_y_generar_encabezados_comunidades(df_nivel_educativo_comunidades)

# Mostrar las primeras filas y la información del DataFrame actualizado
df_nivel_educativo_comunidades.head()



,Código comunidad,Nombre comunidad,Hombres,Mujeres,Argentina,Bolivia,China,Colombia,Cuba,Ecuador,España,Francia,Marruecos,Otros países de América,Otros países de Asia,Otros países de Europa,Otros países de Oceanía,Otros países de África,Perú,Reino Unido,República Dominicana,Rumanía,Ucrania,Venezuela,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,1,Andalucía,14191520,14814188,179896,68788,70360,168684,44220,95272,25504212,109364,694492,309596,97952,608596,5084,204624,53628,340296,34140,254560,65968,95976,5329040,7889300,10066364,5721004,29005708
1,2,Aragón,2257748,2337012,16420,4852,19048,56884,14292,50548,3830992,15676,86312,89092,18628,61128,632,87656,17924,4520,21072,166692,9272,23120,677896,1516724,1391740,1008400,4594760
2,3,"Asturias, Principado de",1703944,1898752,17804,2264,4812,27380,18996,18680,3280668,9356,14500,51244,5660,53372,920,15244,6948,4756,18116,22772,3412,25792,556792,1210520,1035988,799396,3602696
3,4,"Balears, Illes",2017116,2049688,109768,27656,19624,92660,27096,62752,2939664,33368,107764,114968,43276,245684,2108,54416,19192,72096,20548,41612,8608,23944,551496,1146044,1295332,1073932,4066804
4,5,Canarias,3773072,3902360,71592,15352,34976,136776,181500,26268,5980416,27628,100384,98672,51412,429960,1232,61376,17872,113688,18760,29468,6412,271688,1403748,2200628,2267688,1803368,7675432


In [719]:
df_nivel_educativo_comunidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 29 columns):
 #   Column                                                                        Non-Null Count  Dtype 
---  ------                                                                        --------------  ----- 
 0   Código comunidad                                                              19 non-null     int64 
 1   Nombre comunidad                                                              19 non-null     object
 2   Hombres                                                                       19 non-null     int64 
 3   Mujeres                                                                       19 non-null     int64 
 4   Argentina                                                                     19 non-null     int64 
 5   Bolivia                                                                       19 non-null     int64 
 6   China                                       

In [720]:
# Guardamos el DataFrame en un archivo CSV
df_nivel_educativo_comunidades.to_csv("datos_limpiados/comunidades/nivel_educativo_comunidades_limpio.csv", index=False)

## Limpiamos el dataframe con la información del nivel educativo a nivel nacional

In [721]:
df_nivel_educativo_nacional = pd.read_csv("datos/ine/nivel_educativo_nacional.csv", encoding= 'latin-1', sep=';')

In [722]:
df_nivel_educativo_nacional.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,Total,Total,Total,2022,40.899.207
1,Total Nacional,NaN,NaN,Total,Total,Hombres,2022,19.897.392
2,Total Nacional,NaN,NaN,Total,Total,Mujeres,2022,21.001.815
3,Total Nacional,NaN,NaN,Total,Educación primaria e inferior,Total,2022,6.545.911
4,Total Nacional,NaN,NaN,Total,Educación primaria e inferior,Hombres,2022,2.747.314


In [723]:
df_nivel_educativo_nacional.sample(20)

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
162,Total Nacional,NaN,NaN,República Dominicana,Educación superior,Total,2022,28.584
124,Total Nacional,NaN,NaN,Otros países de África,Educación primaria e inferior,Hombres,2022,104.393
180,Total Nacional,NaN,NaN,Bolivia,Total,Total,2022,175.467
300,Total Nacional,NaN,NaN,Otros países de Oceanía,Total,Total,2022,7.967
313,Total Nacional,NaN,NaN,Otros países de Oceanía,Educación superior,Hombres,2022,2.178
195,Total Nacional,NaN,NaN,Colombia,Total,Total,2022,530.954
55,Total Nacional,NaN,NaN,Reino Unido,Segunda etapa de educación secundaria y educac...,Hombres,2022,28.034
236,Total Nacional,NaN,NaN,Perú,Segunda etapa de educación secundaria y educac...,Mujeres,2022,47.121
208,Total Nacional,NaN,NaN,Colombia,Educación superior,Hombres,2022,54.394
140,Total Nacional,NaN,NaN,Cuba,Educación primaria e inferior,Mujeres,2022,16.379


Formateamos la columna 'Total'

In [724]:
# Convertimos la columna "Total" a int64
df_nivel_educativo_nacional['Total'] = df_nivel_educativo_nacional['Total'].str.replace('.', '').astype('int64')
df_nivel_educativo_nacional.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Provincias,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,Total Nacional,NaN,NaN,Total,Total,Total,2022,40899207
1,Total Nacional,NaN,NaN,Total,Total,Hombres,2022,19897392
2,Total Nacional,NaN,NaN,Total,Total,Mujeres,2022,21001815
3,Total Nacional,NaN,NaN,Total,Educación primaria e inferior,Total,2022,6545911
4,Total Nacional,NaN,NaN,Total,Educación primaria e inferior,Hombres,2022,2747314


Limpiamos el dataframe

In [725]:
def procesar_informacion_nacional(df):
    """
    Realiza el procesamiento de un DataFrame con información a nivel nacional, extraída del INE,
    aplicando las siguientes transformaciones:

    1. Elimina las columnas 'Provincias' y 'Comunidades y Ciudades Autónomas'.
    2. Renombra la columna "Total Nacional" a "Nombre del país".
    3. Asigna el valor "España" a todas las filas en la columna "Nombre del país".
        
    Parámetros:
    -----------
        df (pd.DataFrame): DataFrame con la información de nivel educativo a nivel nacional.

    Retorna:
    --------
        pd.DataFrame: DataFrame transformado con las modificaciones aplicadas.
    """
    df = df.copy()  # Crear una copia para no modificar el original
    
    # 1. Eliminar las columnas 'Provincias' y 'Comunidades y Ciudades Autónomas' si existen
    columnas_a_eliminar = ['Comunidades y Ciudades Autónomas', 'Provincias']
    df.drop(columns=[col for col in columnas_a_eliminar if col in df.columns], inplace=True)

    # 2. Renombrar la columna "Total Nacional" a "Nombre del país" si existe
    if "Total Nacional" in df.columns:
        df.rename(columns={"Total Nacional": "Nombre del país"}, inplace=True)

    # 3. Asignar el valor "España" a todas las filas en la columna "Nombre del país"
    if "Nombre del país" in df.columns:
        df["Nombre del país"] = "España"

    return df

# Aplicar la función al DataFrame
df_nivel_educativo_nacional = procesar_informacion_nacional(df_nivel_educativo_nacional)

# Mostrar información y primeras filas del DataFrame actualizado
print(df_nivel_educativo_nacional.info())
df_nivel_educativo_nacional.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Nombre del país               315 non-null    object
 1   País de nacimiento            315 non-null    object
 2   Nivel de formación alcanzado  315 non-null    object
 3   Sexo                          315 non-null    object
 4   Periodo                       315 non-null    int64 
 5   Total                         315 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 14.9+ KB
None


,Nombre del país,País de nacimiento,Nivel de formación alcanzado,Sexo,Periodo,Total
0,España,Total,Total,Total,2022,40899207
1,España,Total,Total,Hombres,2022,19897392
2,España,Total,Total,Mujeres,2022,21001815
3,España,Total,Educación primaria e inferior,Total,2022,6545911
4,España,Total,Educación primaria e inferior,Hombres,2022,2747314


Vamos a pivotar columnas del dataframe para que cada fila contenga toda la información de un país. Generamos encabezados dobles que agrupen la información por categorías.

In [726]:
def pivotar_y_generar_encabezados_nacional(df):
    """
    Transforma un DataFrame de nivel educativo a nivel nacional mediante la pivotación de 
    tres variables clave ('País de nacimiento', 'Nivel de formación alcanzado' y 'Sexo'), 
    y organiza los encabezados con un índice de múltiples niveles.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame que contiene las columnas:
        - 'Nombre del país'
        - 'País de nacimiento'
        - 'Nivel de formación alcanzado'
        - 'Sexo'
        - 'Total'
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame transformado con las columnas pivotadas y organizadas bajo un 
        MultiIndex donde las categorías corresponden a la variable a la que pertenecen.
        Se incluyen los siguientes niveles de encabezado:
        - 'Países' (Código y Nombre del país)
        - 'Sexo' (Hombres y Mujeres)
        - 'País de nacimiento' (Cada país representado en la data)
        - 'Nivel educativo' (Diferentes niveles de formación)
        - 'Población mayor de 15 años' (Total)
    """

    # 1. Verificación de columnas necesarias
    columnas_necesarias = ['Nombre del país', 'País de nacimiento', 'Nivel de formación alcanzado', 'Sexo', 'Total']
    for col in columnas_necesarias:
        if col not in df.columns:
            raise ValueError(f"Falta la columna necesaria en el DataFrame: {col}")

    # 2. Pivotamos la columna "País de nacimiento"
    df_pivotado_pais = df.pivot_table(
        index=['Nombre del país'],
        columns='País de nacimiento',
        values='Total',
        aggfunc='sum'
    ).reset_index()

    # 3. Pivotamos la columna "Nivel de formación alcanzado"
    df_pivotado_nivel = df.pivot_table(
        index=['Nombre del país'],
        columns='Nivel de formación alcanzado',
        values='Total',
        aggfunc='sum'
    ).reset_index()

    # 4. Pivotamos la columna "Sexo"
    df_pivotado_sexo = df.pivot_table(
        index=['Nombre del país'],
        columns='Sexo',
        values='Total',
        aggfunc='sum'
    ).reset_index()

    # 5. Fusionamos los DataFrames resultantes con outer join
    df_concatenado = pd.merge(
        pd.merge(
            df_pivotado_pais, 
            df_pivotado_nivel, 
            on=['Nombre del país'], 
            how='outer'
        ), 
        df_pivotado_sexo, 
        on=['Nombre del país'], 
        how='outer'
    )

    # 6. Eliminamos columnas duplicadas si existen
    columnas_duplicadas = ['Total_x', 'Total_y']
    df_concatenado = df_concatenado.drop(columns=[col for col in columnas_duplicadas if col in df_concatenado.columns], errors='ignore')

    # 7. Renombramos el DataFrame
    df_nivel_educativo_nacional = df_concatenado.copy()

    # 8. Generamos una columna de código de país (España = 1)
    df_nivel_educativo_nacional["Código país"] = 1

    # 9. Reordenamos las columnas
    columnas_reordenadas = (
        ['Código país', 'Nombre del país', 'Hombres', 'Mujeres'] +
        [col for col in df_nivel_educativo_nacional.columns if col not in ['Código país', 'Nombre del país', 'Hombres', 'Mujeres']]
    )
    df_nivel_educativo_nacional = df_nivel_educativo_nacional[columnas_reordenadas]

    # 10. Creamos una lista de categorías para las columnas del DataFrame
    categorias = (
        ['Países'] * 2 +  # Código país, Nombre país
        ['Sexo'] * 2 +  # Hombres, Mujeres
        ['País de nacimiento'] * (len(df_pivotado_pais.columns) - 1) +  # Países de nacimiento
        ['Nivel educativo'] * (len(df_pivotado_nivel.columns) - 1) +  # Niveles educativos
        ['Población mayor de 15 años']  # Total
    )

    # 11. Asignamos las categorías a las columnas del DataFrame
    if len(categorias) == len(df_nivel_educativo_nacional.columns):
        df_nivel_educativo_nacional.columns = pd.MultiIndex.from_tuples(zip(categorias, df_nivel_educativo_nacional.columns))

    return df_nivel_educativo_nacional


# Aplicar la función al DataFrame
df_nivel_educativo_nacional = pivotar_y_generar_encabezados_nacional(df_nivel_educativo_nacional)

# Mostrar las primeras filas del DataFrame actualizado
df_nivel_educativo_nacional.head()


,Código país,Nombre del país,Hombres,Mujeres,Argentina,Bolivia,China,Colombia,Cuba,Ecuador,España,Francia,Marruecos,Otros países de América,Otros países de Asia,Otros países de Europa,Otros países de Oceanía,Otros países de África,Perú,Reino Unido,República Dominicana,Rumanía,Ucrania,Venezuela,Educación primaria e inferior,Educación superior,Primera etapa de educación secundaria y similar,Segunda etapa de educación secundaria y educación postsecundaria no superior,Total
0,1,España,79589568,84007260,1245180,701868,722044,2123816,678776,1642368,135390128,808136,3601748,3052212,1182364,3975804,31868,1445768,997400,1124284,719432,2123684,424240,1605708,26183644,52596496,49384308,35432380,163596828


In [727]:
df_nivel_educativo_nacional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 29 columns):
 #   Column                                                                        Non-Null Count  Dtype 
---  ------                                                                        --------------  ----- 
 0   Código país                                                                   1 non-null      int64 
 1   Nombre del país                                                               1 non-null      object
 2   Hombres                                                                       1 non-null      int64 
 3   Mujeres                                                                       1 non-null      int64 
 4   Argentina                                                                     1 non-null      int64 
 5   Bolivia                                                                       1 non-null      int64 
 6   China                                         

In [728]:
# Guardamos el DataFrame en un archivo CSV
df_nivel_educativo_nacional.to_csv("datos_limpiados/nacionales/nivel_educativo_nacional_limpio.csv", index=False)